# Import libraries

In [281]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb


# Set option to display all columns
pd.set_option('display.max_columns', None)

# Ignore warnings
warnings.filterwarnings('ignore')

# load data

In [282]:
# train data

train_flag = pd.read_csv('data/train/train_flag.csv')

In [283]:
f = open('data/train/enquiry_data_train.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
unq_lst=[]
for val in data:
    unq_lst.extend(val)

train_enquiry = pd.json_normalize(unq_lst)

In [284]:
# train accounts data

f = open('data/train/accounts_data_train.json')
# returns JSON object as a dictionary
data = json.load(f)
unq_lst=[]
for val in data:
    unq_lst.extend(val)
    
train_account = pd.json_normalize(unq_lst)

In [285]:
# strip spances for key
train_flag['uid'] = train_flag['uid'].str.strip()

In [286]:
# change type of TARGET to int8
train_flag.TARGET = train_flag.TARGET.astype('int8')

In [287]:
# Convert enquiry_date to datetime
train_enquiry['enquiry_date'] = pd.to_datetime(train_enquiry['enquiry_date'])

In [ ]:
# test data
test_flag = pd.read_csv('data/test/test_flag.csv')

# train accounts data
f = open('data/test/accounts_data_test.json')
# returns JSON object as a dictionary
data = json.load(f)
unq_lst=[]
for val in data:
    unq_lst.extend(val)
    
test_account = pd.json_normalize(unq_lst)


f = open('data/test/enquiry_data_test.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
unq_lst=[]
for val in data:
    unq_lst.extend(val)

test_enquiry = pd.json_normalize(unq_lst)


# Exploring uids whose data is in flag but not in account

In [288]:
# Ensure unique uids
unique_accounts_uids = train_account['uid'].unique()
unique_flag_uids = train_flag['uid'].unique()

# Find UIDs in train_flag but not in train_account
uids_in_flag_not_in_account = train_flag[~train_flag['uid'].isin(train_account['uid'])]

In [289]:
uids_in_flag_not_in_account

,uid,NAME_CONTRACT_TYPE,TARGET
4,WJZ68772744,Cash loans,0
7,DBK56652751,Cash loans,1
13,VJY65237599,Cash loans,0
25,CDR05672029,Cash loans,0
27,GYL79147955,Cash loans,1
...,...,...,...
261354,BZF21478094,Cash loans,1
261357,NKC15771473,Cash loans,0
261361,PJK40550802,Cash loans,0
261376,RJY83457377,Cash loans,0


In [290]:
uids_in_flag_not_in_account.TARGET.value_counts(normalize=True)*100

TARGET
0    89.8572
1    10.1428
Name: proportion, dtype: float64

**lets have a sperate model for data where accoutns is missing**

# Feature engineering enquiry
aggregation and merge

In [331]:
# Convert enquiry_date to datetime if not already
train_enquiry['enquiry_date'] = pd.to_datetime(train_enquiry['enquiry_date'])


def days_since_last_enquiry(dates):
    dates_sorted = dates.sort_values(ascending=False)
    if len(dates_sorted) > 1:
        return (dates_sorted.iloc[0] - dates_sorted.iloc[1]).days
    else:
        return 0  # or any other value to indicate insufficient data

def days_since_first_enquiry(dates):
    return (dates.max() - dates.min()).days

def count_enquiries_in_last_days(dates, days):
    recent_date = dates.max()
    return (recent_date - dates).dt.days.le(days).sum()

# Aggregation by uid
aggregated_enquiry_df = train_enquiry.groupby('uid').agg(
    enquiry_count=('enquiry_amt', 'count'),
    avg_enquiry_amt=('enquiry_amt', 'mean'),
    total_enquiry_amt=('enquiry_amt', 'sum'),
    recent_enquiry_date=('enquiry_date', 'max'),
    days_since_last_enquiry=('enquiry_date', days_since_last_enquiry),
    most_recent_enquiry_amt=('enquiry_amt', lambda x: x.loc[x.idxmax()]),
    min_enquiry_amt=('enquiry_amt', 'min'),
    max_enquiry_amt=('enquiry_amt', 'max'),
    std_enquiry_amt=('enquiry_amt', 'std'),
    days_since_first_enquiry=('enquiry_date', days_since_first_enquiry),
    avg_days_between_enquiries=('enquiry_date', lambda x: x.diff().mean().days if len(x) > 1 else 0),
    recent_enquiry_type=('enquiry_type', 'last')
)

# Calculate counts of enquiries in the last 10 and 30 days
enquiry_counts = train_enquiry.groupby('uid')['enquiry_date'].apply(
    lambda x: pd.Series({
        'enquiry_count_10': count_enquiries_in_last_days(x, 10),
        'enquiry_count_30': count_enquiries_in_last_days(x, 30)
    })
).unstack()

# Merge with aggregated data
aggregated_enquiry_df = aggregated_enquiry_df.merge(enquiry_counts, on='uid', how='left')

# Merge with enquiry type counts
enquiry_type_counts = train_enquiry.groupby(['uid', 'enquiry_type']).size().unstack(fill_value=0)
enquiry_type_counts.columns = [f'enquiry_type_{col}_count' for col in enquiry_type_counts.columns]

aggregated_enquiry_df = aggregated_enquiry_df.merge(enquiry_type_counts, on='uid', how='left')

# Reset index to make uid a column again
aggregated_enquiry_df = aggregated_enquiry_df.reset_index()

In [332]:
enquiry_counts

,enquiry_count_10,enquiry_count_30
uid,,
AAA08065248,1,1
AAA09044550,1,3
AAA10545297,1,1
AAA14112888,3,3
AAA20326915,1,1
...,...,...
ZZZ74526004,1,1
ZZZ78449185,1,1
ZZZ79008454,1,1


In [333]:
aggregated_enquiry_df.enquiry_count_10

0         1
1         1
2         1
3         3
4         1
         ..
261378    1
261379    1
261380    1
261381    1
261382    1
Name: enquiry_count_10, Length: 261383, dtype: int64

In [334]:
aggregated_enquiry_df.head()

,uid,enquiry_count,avg_enquiry_amt,total_enquiry_amt,recent_enquiry_date,days_since_last_enquiry,most_recent_enquiry_amt,min_enquiry_amt,max_enquiry_amt,std_enquiry_amt,days_since_first_enquiry,avg_days_between_enquiries,recent_enquiry_type,enquiry_count_10,enquiry_count_30,enquiry_type_Another type of loan_count,enquiry_type_Car loan_count,enquiry_type_Cash loan (non-earmarked)_count,enquiry_type_Cash loans_count,enquiry_type_Consumer credit_count,enquiry_type_Credit card_count,enquiry_type_Interbank credit_count,enquiry_type_Loan for business development_count,enquiry_type_Loan for purchase of shares (margin lending)_count,enquiry_type_Loan for the purchase of equipment_count,enquiry_type_Loan for working capital replenishment_count,enquiry_type_Microloan_count,enquiry_type_Mobile operator loan_count,enquiry_type_Mortgage_count,enquiry_type_Real estate loan_count,enquiry_type_Revolving loans_count,enquiry_type_Unknown type of loan_count
0,AAA08065248,11,187696.181818,2064658,2020-12-30,52,364751,36082,364751,102098.260115,670,5,Loan for purchase of shares (margin lending),1,1,0,0,1,0,0,0,3,0,2,0,0,0,2,1,0,2,0
1,AAA09044550,26,102269.230769,2659000,2020-12-29,18,197000,5000,197000,50263.750511,1169,-4,Cash loans,1,3,3,3,1,7,1,1,2,0,0,1,0,2,1,1,1,1,1
2,AAA10545297,14,94071.428571,1317000,2020-10-29,32,192000,5000,192000,66014.525541,846,-23,Credit card,1,1,1,0,2,2,2,2,1,0,1,0,0,0,0,0,0,2,1
3,AAA14112888,15,97666.666667,1465000,2020-07-05,4,185000,17000,185000,49185.750937,1179,0,Loan for working capital replenishment,3,3,1,0,1,1,2,0,1,0,2,2,2,0,2,0,0,0,1
4,AAA20326915,1,66000.000000,66000,2020-08-14,0,66000,66000,66000,NaN,0,0,Mobile operator loan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [293]:
train_flag[train_flag.uid=='AAA08065248']

,uid,NAME_CONTRACT_TYPE,TARGET
160539,AAA08065248,Revolving loans,0


In [335]:
aggregated_enquiry_df.isna().sum()

uid                                                                    0
enquiry_count                                                          0
avg_enquiry_amt                                                        0
total_enquiry_amt                                                      0
recent_enquiry_date                                                    0
days_since_last_enquiry                                                0
most_recent_enquiry_amt                                                0
min_enquiry_amt                                                        0
max_enquiry_amt                                                        0
std_enquiry_amt                                                    56211
days_since_first_enquiry                                               0
avg_days_between_enquiries                                             0
recent_enquiry_type                                                    0
enquiry_count_10                                   

In [336]:
aggregated_enquiry_df = aggregated_enquiry_df.drop('std_enquiry_amt',axis=1)

In [337]:
aggregated_enquiry_df.shape

(261383, 31)

In [338]:
train_flag.shape

(261383, 3)

In [339]:
final_df = pd.merge(aggregated_enquiry_df, train_flag, on='uid', how='inner')

In [340]:
final_df.shape

(261383, 33)

In [341]:
final_df.head()

,uid,enquiry_count,avg_enquiry_amt,total_enquiry_amt,recent_enquiry_date,days_since_last_enquiry,most_recent_enquiry_amt,min_enquiry_amt,max_enquiry_amt,days_since_first_enquiry,avg_days_between_enquiries,recent_enquiry_type,enquiry_count_10,enquiry_count_30,enquiry_type_Another type of loan_count,enquiry_type_Car loan_count,enquiry_type_Cash loan (non-earmarked)_count,enquiry_type_Cash loans_count,enquiry_type_Consumer credit_count,enquiry_type_Credit card_count,enquiry_type_Interbank credit_count,enquiry_type_Loan for business development_count,enquiry_type_Loan for purchase of shares (margin lending)_count,enquiry_type_Loan for the purchase of equipment_count,enquiry_type_Loan for working capital replenishment_count,enquiry_type_Microloan_count,enquiry_type_Mobile operator loan_count,enquiry_type_Mortgage_count,enquiry_type_Real estate loan_count,enquiry_type_Revolving loans_count,enquiry_type_Unknown type of loan_count,NAME_CONTRACT_TYPE,TARGET
0,AAA08065248,11,187696.181818,2064658,2020-12-30,52,364751,36082,364751,670,5,Loan for purchase of shares (margin lending),1,1,0,0,1,0,0,0,3,0,2,0,0,0,2,1,0,2,0,Revolving loans,0
1,AAA09044550,26,102269.230769,2659000,2020-12-29,18,197000,5000,197000,1169,-4,Cash loans,1,3,3,3,1,7,1,1,2,0,0,1,0,2,1,1,1,1,1,Cash loans,0
2,AAA10545297,14,94071.428571,1317000,2020-10-29,32,192000,5000,192000,846,-23,Credit card,1,1,1,0,2,2,2,2,1,0,1,0,0,0,0,0,0,2,1,Cash loans,0
3,AAA14112888,15,97666.666667,1465000,2020-07-05,4,185000,17000,185000,1179,0,Loan for working capital replenishment,3,3,1,0,1,1,2,0,1,0,2,2,2,0,2,0,0,0,1,Cash loans,0
4,AAA20326915,1,66000.000000,66000,2020-08-14,0,66000,66000,66000,0,0,Mobile operator loan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Cash loans,0


# Feature engineering account

In [342]:
train_account.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,consumer_credit,credit_card,others,payment_hist,max_days_overdue,min_days_overdue,avg_days_overdue,count_overdue_months,most_recent_payment_status,loan_duration
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",10,0,0.588235,1,0,518
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,1,0,0,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",44,0,3.625000,2,0,504
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,0,1,0,"[0, 0, 0, 0, 0, 0]",0,0,0.000000,0,0,186
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,1,0,0,"[0, 0, 0]",0,0,0.000000,0,0,92
4,Credit card,10480.500,0.0,2014-09-10,NaT,0000000000000000000000000000000000000000000000...,AAA20326915,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",917,0,190.486842,31,0,2305


In [343]:
# make credit type into three categories

In [344]:
train_account.credit_type.value_counts()

credit_type
Consumer credit                                 908741
Credit card                                     292422
Car loan                                         20237
Mortgage                                         13196
Microloan                                         7766
Loan for business development                     1466
Another type of loan                               732
Unknown type of loan                               374
Loan for working capital replenishment             320
Real estate loan                                    17
Cash loan (non-earmarked)                           17
Loan for the purchase of equipment                  16
Loan for purchase of shares (margin lending)         4
Interbank credit                                     1
Mobile operator loan                                 1
Name: count, dtype: int64

In [345]:
# Ensure 'closed_date' and 'open_date' are datetime
train_account['open_date'] = pd.to_datetime(train_account['open_date'])
train_account['closed_date'] = pd.to_datetime(train_account['closed_date'])

# Create binary columns for credit types
train_account['consumer_credit'] = (train_account['credit_type'] == 'Consumer credit').astype(int)
train_account['credit_card'] = (train_account['credit_type'] == 'Credit card').astype(int)
train_account['others'] = (~train_account['credit_type'].isin(['Consumer credit', 'Credit card'])).astype(int)

# Handle payment_hist_string
def process_payment_hist_string(s):
    if pd.isna(s):
        return []
    return [int(s[i:i+3]) for i in range(0, len(s), 3)]

train_account['payment_hist'] = train_account['payment_hist_string'].apply(process_payment_hist_string)

# Feature extraction from payment_hist
train_account['max_days_overdue'] = train_account['payment_hist'].apply(lambda x: max(x) if x else 0)
train_account['min_days_overdue'] = train_account['payment_hist'].apply(lambda x: min(x) if x else 0)
train_account['avg_days_overdue'] = train_account['payment_hist'].apply(lambda x: np.mean(x) if x else 0)
train_account['count_overdue_months'] = train_account['payment_hist'].apply(lambda x: sum(1 for i in x if i > 0))
train_account['most_recent_payment_status'] = train_account['payment_hist'].apply(lambda x: x[-1] if x else 0)

max_date = final_df['recent_enquiry_date'].max()

# Ensure 'open_date' and 'closed_date' are datetime
train_account['open_date'] = pd.to_datetime(train_account['open_date'])
train_account['closed_date'] = pd.to_datetime(train_account['closed_date'])

# Calculate loan duration
train_account['loan_duration'] = (train_account['closed_date'].fillna(max_date) - train_account['open_date']).dt.days

total_loan_amount = train_account['loan_amount'].sum()

def proportion_overdue(x):
    return x.sum() / max(1, total_loan_amount)

def loan_status_ratio(x):
    return x.isna().sum() / max(1, x.notna().sum())

aggregated_account_df = train_account.groupby('uid').agg(
    total_loan_amount=('loan_amount', 'sum'),
    avg_loan_amount=('loan_amount', 'mean'),
    max_loan_amount=('loan_amount', 'max'),
    total_amount_overdue=('amount_overdue', 'sum'),
    avg_amount_overdue=('amount_overdue', 'mean'),
    max_amount_overdue=('amount_overdue', 'max'),
    proportion_overdue=('amount_overdue', proportion_overdue),
    ongoing_loan_amount=('loan_amount', lambda x: x[train_account.loc[x.index, 'closed_date'].isna()].sum()),
    closed_loan_amount=('loan_amount', lambda x: x[train_account.loc[x.index, 'closed_date'].notna()].sum()),
    ongoing_loan_count=('closed_date', lambda x: x.isna().sum()),
    closed_loan_count=('closed_date', lambda x: x.notna().sum()),
    loan_status_ratio=('closed_date', loan_status_ratio),
    avg_loan_duration=('loan_duration', 'mean'),
    max_loan_duration=('loan_duration', 'max'),
    min_loan_duration=('loan_duration', 'min'),
    loan_type_diversity=('credit_type', 'nunique'),
    max_days_overdue=('max_days_overdue', 'max'),
    min_days_overdue=('min_days_overdue', 'min'),
    avg_days_overdue=('avg_days_overdue', 'mean'),
    count_overdue_months=('count_overdue_months', 'sum'),
    most_recent_payment_status=('most_recent_payment_status', 'max'),
    consumer_credit_count=('consumer_credit', 'sum'),
    credit_card_count=('credit_card', 'sum'),
    other_loans_count=('others', 'sum')
).reset_index()

print(aggregated_account_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223918 entries, 0 to 223917
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   uid                         223918 non-null  object 
 1   total_loan_amount           223918 non-null  float64
 2   avg_loan_amount             223917 non-null  float64
 3   max_loan_amount             223917 non-null  float64
 4   total_amount_overdue        223918 non-null  float64
 5   avg_amount_overdue          223918 non-null  float64
 6   max_amount_overdue          223918 non-null  float64
 7   proportion_overdue          223918 non-null  float64
 8   ongoing_loan_amount         223918 non-null  float64
 9   closed_loan_amount          223918 non-null  float64
 10  ongoing_loan_count          223918 non-null  int64  
 11  closed_loan_count           223918 non-null  int64  
 12  loan_status_ratio           223918 non-null  float64
 13  avg_loan_durat

In [346]:
# # Create binary columns for credit types
# train_account['consumer_credit'] = (train_account['credit_type'] == 'Consumer credit').astype(int)
# train_account['credit_card'] = (train_account['credit_type'] == 'Credit card').astype(int)
# train_account['others'] = (~train_account['credit_type'].isin(['Consumer credit', 'Credit card'])).astype(int)

# # Handle payment_hist_string
# def process_payment_hist_string(s):
#     if pd.isna(s):
#         return []
#     return [int(s[i:i+3]) for i in range(0, len(s), 3)]

# train_account['payment_hist'] = train_account['payment_hist_string'].apply(process_payment_hist_string)

# # Feature extraction from payment_hist
# train_account['max_days_overdue'] = train_account['payment_hist'].apply(lambda x: max(x) if x else 0)
# train_account['min_days_overdue'] = train_account['payment_hist'].apply(lambda x: min(x) if x else 0)
# train_account['avg_days_overdue'] = train_account['payment_hist'].apply(lambda x: np.mean(x) if x else 0)
# train_account['count_overdue_months'] = train_account['payment_hist'].apply(lambda x: sum(1 for i in x if i > 0))
# train_account['most_recent_payment_status'] = train_account['payment_hist'].apply(lambda x: x[-1] if x else 0)

# # Calculate loan duration
# train_account['loan_duration'] = (pd.to_datetime(train_account['closed_date']) - pd.to_datetime(train_account['open_date'])).dt.days
# train_account['loan_duration'] = train_account['loan_duration'].fillna(0)  # Fill NaN for ongoing loans

In [347]:
# total_loan_amount = train_account['loan_amount'].sum()

In [348]:
# def proportion_overdue(x):
#     return x.sum() / max(1, total_loan_amount)  # Use the precomputed total_loan_amount

# def loan_status_ratio(x):
#     return x.isna().sum() / max(1, x.notna().sum())  # Ratio of ongoing to closed

In [349]:
# aggregated_account_df = train_account.groupby('uid').agg(
#     total_loan_amount=('loan_amount', 'sum'),
#     avg_loan_amount=('loan_amount', 'mean'),
#     max_loan_amount=('loan_amount', 'max'),
#     total_amount_overdue=('amount_overdue', 'sum'),
#     avg_amount_overdue=('amount_overdue', 'mean'),
#     max_amount_overdue=('amount_overdue', 'max'),
#     proportion_overdue=('amount_overdue', proportion_overdue),  # Use the efficient proportion_overdue function
#     ongoing_loan_count=('closed_date', lambda x: x.isna().sum()),
#     closed_loan_count=('closed_date', lambda x: x.notna().sum()),
#     loan_status_ratio=('closed_date', loan_status_ratio),  # Use the efficient loan_status_ratio function
#     avg_loan_duration=('loan_duration', 'mean'),
#     max_loan_duration=('loan_duration', 'max'),
#     min_loan_duration=('loan_duration', 'min'),
#     loan_type_diversity=('credit_type', 'nunique'),
#     max_days_overdue=('max_days_overdue', 'max'),
#     min_days_overdue=('min_days_overdue', 'min'),
#     avg_days_overdue=('avg_days_overdue', 'mean'),
#     count_overdue_months=('count_overdue_months', 'sum'),
#     most_recent_payment_status=('most_recent_payment_status', 'max'),
#     consumer_credit_count=('consumer_credit', 'sum'),
#     credit_card_count=('credit_card', 'sum'),
#     other_loans_count=('others', 'sum')  # Ensure 'others' column exists
# ).reset_index()


# print(aggregated_account_df.info())

In [350]:
aggregated_account_df.sample(5)

,uid,total_loan_amount,avg_loan_amount,max_loan_amount,total_amount_overdue,avg_amount_overdue,max_amount_overdue,proportion_overdue,ongoing_loan_amount,closed_loan_amount,ongoing_loan_count,closed_loan_count,loan_status_ratio,avg_loan_duration,max_loan_duration,min_loan_duration,loan_type_diversity,max_days_overdue,min_days_overdue,avg_days_overdue,count_overdue_months,most_recent_payment_status,consumer_credit_count,credit_card_count,other_loans_count
111782,MZR09524361,1139230.485,284807.62125,630000.0,0.0,0.0,0.0,0.0,1139230.485,0.0,4,0,4.000000,616.0,1272,97,2,0,0,0.000000,0,0,3,1,0
54161,GHL39383970,257310.000,85770.00000,180000.0,0.0,0.0,0.0,0.0,180000.000,77310.0,1,2,0.500000,352.0,489,122,2,0,0,0.000000,0,0,1,2,0
36756,EHJ90569195,360000.000,360000.00000,360000.0,0.0,0.0,0.0,0.0,0.000,360000.0,0,1,0.000000,1811.0,1811,1811,1,0,0,0.000000,0,0,1,0,0
91070,KPE71617040,5136763.500,642095.43750,1665000.0,0.0,0.0,0.0,0.0,3516763.500,1620000.0,5,3,1.666667,376.0,540,34,2,35,0,0.481618,3,0,5,3,0
136538,PVM84921969,68841.000,34420.50000,34420.5,0.0,0.0,0.0,0.0,0.000,68841.0,0,2,0.000000,297.0,357,237,1,0,0,0.000000,0,0,2,0,0


In [351]:
aggregated_account_df.total_amount_overdue.value_counts()

total_amount_overdue
0.00         221089
4.50            226
9.00             77
13.50            60
22.50            48
              ...  
1503.00           1
89.64             1
627.84            1
21447.00          1
127260.00         1
Name: count, Length: 1077, dtype: int64

In [352]:
df = pd.merge(aggregated_account_df, final_df, on='uid', how='right')

In [353]:
df.head()

,uid,total_loan_amount,avg_loan_amount,max_loan_amount,total_amount_overdue,avg_amount_overdue,max_amount_overdue,proportion_overdue,ongoing_loan_amount,closed_loan_amount,ongoing_loan_count,closed_loan_count,loan_status_ratio,avg_loan_duration,max_loan_duration,min_loan_duration,loan_type_diversity,max_days_overdue,min_days_overdue,avg_days_overdue,count_overdue_months,most_recent_payment_status,consumer_credit_count,credit_card_count,other_loans_count,enquiry_count,avg_enquiry_amt,total_enquiry_amt,recent_enquiry_date,days_since_last_enquiry,most_recent_enquiry_amt,min_enquiry_amt,max_enquiry_amt,days_since_first_enquiry,avg_days_between_enquiries,recent_enquiry_type,enquiry_count_10,enquiry_count_30,enquiry_type_Another type of loan_count,enquiry_type_Car loan_count,enquiry_type_Cash loan (non-earmarked)_count,enquiry_type_Cash loans_count,enquiry_type_Consumer credit_count,enquiry_type_Credit card_count,enquiry_type_Interbank credit_count,enquiry_type_Loan for business development_count,enquiry_type_Loan for purchase of shares (margin lending)_count,enquiry_type_Loan for the purchase of equipment_count,enquiry_type_Loan for working capital replenishment_count,enquiry_type_Microloan_count,enquiry_type_Mobile operator loan_count,enquiry_type_Mortgage_count,enquiry_type_Real estate loan_count,enquiry_type_Revolving loans_count,enquiry_type_Unknown type of loan_count,NAME_CONTRACT_TYPE,TARGET
0,AAA08065248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,187696.181818,2064658,2020-12-30,52,364751,36082,364751,670,5,Loan for purchase of shares (margin lending),1,1,0,0,1,0,0,0,3,0,2,0,0,0,2,1,0,2,0,Revolving loans,0
1,AAA09044550,277245.000,138622.500,272745.000,0.0,0.0,0.0,0.0,0.000,277245.0,0.0,2.0,0.0,511.00,518.0,504.0,1.0,44.0,0.0,2.106618,3.0,0.0,2.0,0.0,0.0,26,102269.230769,2659000,2020-12-29,18,197000,5000,197000,1169,-4,Cash loans,1,3,3,3,1,7,1,1,2,0,0,1,0,2,1,1,1,1,1,Cash loans,0
2,AAA10545297,80996.445,80996.445,80996.445,0.0,0.0,0.0,0.0,80996.445,0.0,1.0,0.0,1.0,186.00,186.0,186.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,14,94071.428571,1317000,2020-10-29,32,192000,5000,192000,846,-23,Credit card,1,1,1,0,2,2,2,2,1,0,1,0,0,0,0,0,0,2,1,Cash loans,0
3,AAA14112888,43771.500,43771.500,43771.500,0.0,0.0,0.0,0.0,0.000,43771.5,0.0,1.0,0.0,92.00,92.0,92.0,1.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,15,97666.666667,1465000,2020-07-05,4,185000,17000,185000,1179,0,Loan for working capital replenishment,3,3,1,0,1,1,2,0,1,0,2,2,2,0,2,0,0,0,1,Cash loans,0
4,AAA20326915,591597.000,73949.625,235800.000,0.0,0.0,0.0,0.0,102100.500,489496.5,3.0,5.0,0.6,577.75,2305.0,31.0,2.0,917.0,0.0,23.810855,31.0,0.0,5.0,3.0,0.0,1,66000.000000,66000,2020-08-14,0,66000,66000,66000,0,0,Mobile operator loan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Cash loans,0


In [354]:
df.shape

(261383, 57)

In [355]:
final_df.shape

(261383, 33)

In [356]:
df['recent_enquiry_date'] = pd.to_datetime(df['recent_enquiry_date'])
df['enquiry_year'] = df['recent_enquiry_date'].dt.year
df['enquiry_month'] = df['recent_enquiry_date'].dt.month

In [357]:
df = pd.get_dummies(df, columns=['recent_enquiry_type'], prefix='enquiry_type_hot')
# Identify one-hot encoded columns
dummies_columns = [col for col in df.columns if col.startswith('enquiry_type_hot')]

# Convert those columns to integers
df[dummies_columns] = df[dummies_columns].astype(int)

In [358]:
df.NAME_CONTRACT_TYPE.replace({'Cash loans':0, 'Revolving loans':1}, inplace=True)

In [359]:
df.head()

,uid,total_loan_amount,avg_loan_amount,max_loan_amount,total_amount_overdue,avg_amount_overdue,max_amount_overdue,proportion_overdue,ongoing_loan_amount,closed_loan_amount,ongoing_loan_count,closed_loan_count,loan_status_ratio,avg_loan_duration,max_loan_duration,min_loan_duration,loan_type_diversity,max_days_overdue,min_days_overdue,avg_days_overdue,count_overdue_months,most_recent_payment_status,consumer_credit_count,credit_card_count,other_loans_count,enquiry_count,avg_enquiry_amt,total_enquiry_amt,recent_enquiry_date,days_since_last_enquiry,most_recent_enquiry_amt,min_enquiry_amt,max_enquiry_amt,days_since_first_enquiry,avg_days_between_enquiries,enquiry_count_10,enquiry_count_30,enquiry_type_Another type of loan_count,enquiry_type_Car loan_count,enquiry_type_Cash loan (non-earmarked)_count,enquiry_type_Cash loans_count,enquiry_type_Consumer credit_count,enquiry_type_Credit card_count,enquiry_type_Interbank credit_count,enquiry_type_Loan for business development_count,enquiry_type_Loan for purchase of shares (margin lending)_count,enquiry_type_Loan for the purchase of equipment_count,enquiry_type_Loan for working capital replenishment_count,enquiry_type_Microloan_count,enquiry_type_Mobile operator loan_count,enquiry_type_Mortgage_count,enquiry_type_Real estate loan_count,enquiry_type_Revolving loans_count,enquiry_type_Unknown type of loan_count,NAME_CONTRACT_TYPE,TARGET,enquiry_year,enquiry_month,enquiry_type_hot_Another type of loan,enquiry_type_hot_Car loan,enquiry_type_hot_Cash loan (non-earmarked),enquiry_type_hot_Cash loans,enquiry_type_hot_Consumer credit,enquiry_type_hot_Credit card,enquiry_type_hot_Interbank credit,enquiry_type_hot_Loan for business development,enquiry_type_hot_Loan for purchase of shares (margin lending),enquiry_type_hot_Loan for the purchase of equipment,enquiry_type_hot_Loan for working capital replenishment,enquiry_type_hot_Microloan,enquiry_type_hot_Mobile operator loan,enquiry_type_hot_Mortgage,enquiry_type_hot_Real estate loan,enquiry_type_hot_Revolving loans,enquiry_type_hot_Unknown type of loan
0,AAA08065248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,187696.181818,2064658,2020-12-30,52,364751,36082,364751,670,5,1,1,0,0,1,0,0,0,3,0,2,0,0,0,2,1,0,2,0,1,0,2020,12,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,AAA09044550,277245.000,138622.500,272745.000,0.0,0.0,0.0,0.0,0.000,277245.0,0.0,2.0,0.0,511.00,518.0,504.0,1.0,44.0,0.0,2.106618,3.0,0.0,2.0,0.0,0.0,26,102269.230769,2659000,2020-12-29,18,197000,5000,197000,1169,-4,1,3,3,3,1,7,1,1,2,0,0,1,0,2,1,1,1,1,1,0,0,2020,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AAA10545297,80996.445,80996.445,80996.445,0.0,0.0,0.0,0.0,80996.445,0.0,1.0,0.0,1.0,186.00,186.0,186.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,14,94071.428571,1317000,2020-10-29,32,192000,5000,192000,846,-23,1,1,1,0,2,2,2,2,1,0,1,0,0,0,0,0,0,2,1,0,0,2020,10,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,AAA14112888,43771.500,43771.500,43771.500,0.0,0.0,0.0,0.0,0.000,43771.5,0.0,1.0,0.0,92.00,92.0,92.0,1.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,15,97666.666667,1465000,2020-07-05,4,185000,17000,185000,1179,0,3,3,1,0,1,1,2,0,1,0,2,2,2,0,2,0,0,0,1,0,0,2020,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,AAA20326915,591597.000,73949.625,235800.000,0.0,0.0,0.0,0.0,102100.500,489496.5,3.0,5.0,0.6,577.75,2305.0,31.0,2.0,917.0,0.0,23.810855,31.0,0.0,5.0,3.0,0.0,1,66000.000000,66000,2020-08-14,0,66000,66000,66000,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2020,8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [360]:
# seperate data for where account data is available

In [361]:
df1 = df[~df.total_loan_amount.isna()]

In [362]:
df1.shape

(223918, 75)

In [363]:
df2 = df[df.total_loan_amount.isna()]

In [364]:
df2.shape

(37465, 75)

In [365]:
# data clean up and rest of the feature engineering

In [366]:
df1.to_csv('model_with_account.csv')

In [367]:
df2.to_csv('model_wo_account.csv')